In [1]:
from appgeopy import *
from my_packages import *

In [2]:
# ------------------------------------------------------------------------------
# Load Configuration
# ------------------------------------------------------------------------------
config_fpath = (
    r"D:\VENV_PYTHON\insar_scriptflow\C__insstackpsi_calibration_config.ini"
)
config = configparser.ConfigParser()
config.read(config_fpath)

# Extract general parameters from the configuration file
workflow_step = config["GENERAL"]["workflow_step"]
mainfolder = config["GENERAL"]["insstackpsi_mainfolder"]
gps_shapefile_fpath = config["GENERAL"]["gps_shapefile"]

# Define paths for post-processing folders
post_insstackpsi_fld = os.path.join(mainfolder, "14_POST-PROCESSING")
config_fld = os.path.join(post_insstackpsi_fld, "configs")
los_difference_output = config["GPS_DIFFERENCE"]["los_difference_output"]

# Determine output folder based on the workflow step
if workflow_step == "calibration":
    calibration_fld = os.path.join(mainfolder, "15_CALIBRATION")
    prepared_los_difference_outname = "PREPARED_" + los_difference_output
    os.makedirs(calibration_fld, exist_ok=True)

# Define path for LOS difference file
los_difference_fpath = os.path.join(post_insstackpsi_fld, los_difference_output)

# ------------------------------------------------------------------------------
# Load Sheet Names
# ------------------------------------------------------------------------------
# Attempt to load the sheet names from the Excel file
try:
    available_sheetnames = get_sheetnames(los_difference_fpath)
except FileNotFoundError as e:
    print(
        f"Error: The file '{los_difference_fpath}' was not found. Please check the path and try again."
    )
    raise
except Exception as e:
    print(f"Error loading sheet names from '{los_difference_fpath}': {e}")
    raise

# ------------------------------------------------------------------------------
# Load GPS Stations Shapefile
# ------------------------------------------------------------------------------
# Attempt to read the GPS stations shapefile
try:
    gps_shapefile_geodf = gpd.read_file(gps_shapefile_fpath)
    print(f"Loaded GPS shapefile from {gps_shapefile_fpath}")
except Exception as e:
    print(f"Error loading GPS shapefile: {e}")
    raise

# ------------------------------------------------------------------------------
# Prepare DataFrame for All Stations
# ------------------------------------------------------------------------------
# Initialize an empty DataFrame to store LOS measurements for all stations
los_meas_allstations = pd.DataFrame({"STATION": available_sheetnames})

# Extract X and Y coordinates for each station from the shapefile
los_meas_allstations["X_TWD97"] = los_meas_allstations["STATION"].apply(
    lambda station: gps_shapefile_geodf.query(f"STATION == '{station}'")[
        "X_TWD97"
    ].item()
)
los_meas_allstations["Y_TWD97"] = los_meas_allstations["STATION"].apply(
    lambda station: gps_shapefile_geodf.query(f"STATION == '{station}'")[
        "Y_TWD97"
    ].item()
)

# Set the index of the DataFrame to the station names
los_meas_allstations = los_meas_allstations.set_index("STATION")

# ------------------------------------------------------------------------------
# Determine Standard Time Index
# ------------------------------------------------------------------------------
# Find the DataFrame with the maximum number of rows (standard time index)
df_bySheet_maxlength = max(
    pd.read_excel(los_difference_fpath, sheet_name=sheet).shape[0]
    for sheet in available_sheetnames
)

# Loop through the sheets to find the one with the maximum length to set as standard time index
standard_timeidx = None
for sheet in available_sheetnames:
    df_bySheet = pd.read_excel(
        los_difference_fpath, sheet_name=sheet, index_col=[0], parse_dates=[0]
    )
    if len(df_bySheet) == df_bySheet_maxlength:
        standard_timeidx = df_bySheet.index
        break

# Initialize a DataFrame with the standard time index
temp = pd.DataFrame(index=standard_timeidx)

# ------------------------------------------------------------------------------
# Map LOS Values to Standard Time Index
# ------------------------------------------------------------------------------
# Loop through each sheet and map LOS values to the standard time index
for sheet in available_sheetnames:
    df_bySheet = pd.read_excel(
        los_difference_fpath, sheet_name=sheet, index_col=[0], parse_dates=[0]
    )
    temp[sheet] = temp.index.map(df_bySheet["LOS(mm)"])

# Transpose the DataFrame and rename columns to include dates
temp_trans = temp.T
temp_trans.columns = [
    "N" + col.strftime("%Y%m%d") for col in temp_trans.columns
]

# Map the transformed LOS values to the main DataFrame for all stations
for col in temp_trans.columns:
    los_meas_allstations[col] = los_meas_allstations.index.map(temp_trans[col])

# Reset the index of the DataFrame
los_meas_allstations = los_meas_allstations.reset_index()

# ------------------------------------------------------------------------------
# Save the Final DataFrame to an Excel File
# ------------------------------------------------------------------------------
# Define the output path for the final DataFrame and save it to an Excel file
output_path = os.path.join(calibration_fld, prepared_los_difference_outname)
los_meas_allstations.to_excel(output_path, index=False)

print(f"Processed data saved to {output_path}")

Loaded GPS shapefile from D:\1000_SCRIPTS\002_PostQE_Scripts\20240527_GPS_Data\20240605\Shapefiles\2023_NewGPS_Locations_TWD97.shp
Processed data saved to E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\15_CALIBRATION\PREPARED_InSAR_GPS_LOS_Difference.xlsx
